In [12]:
import numpy as np
from sklearn import preprocessing
numofDrug = 708
numofProtein = 1512
featureFusion = list()
featureLable = list()

In [13]:
#def featureFusion(featureFile1,featureFile2,interactionFile):
feature1 = np.loadtxt("DrugFeature.txt")
feature1Norm = preprocessing.normalize(feature1,axis = 0)
feature2 = np.loadtxt("ProteinFeature.txt")
feature2Norm = preprocessing.normalize(feature2,axis = 0)
interaction = np.loadtxt("mat_drug_protein.txt")
np.shape(interaction)  
for i in range(numofDrug):
    for j in range(numofProtein):
        featureFusion.append(np.concatenate((feature1Norm[i],feature2Norm[j]),axis = None))
        featureLable.append(interaction[i][j])len(featureFusion)
len(featureLable)

1070496

In [ ]:
np.savetxt("featureFusion.txt",featureFusion)
np.savetxt("featureLable.txt",featureLable)